In [1]:
from gensim.utils import tokenize
from langdetect import detect
import string
from itertools import islice
import numpy as np
from scipy.sparse import csr_matrix
import smart_open
import faiss
import pandas as pd

import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.linalg import Vectors, VectorUDT
from collections import defaultdict

import pickle
import graphene

import os
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import numpy as np
import ssl
from elasticsearch.connection import create_ssl_context
from elasticsearch_dsl import Q

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1568977756463_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
class Company(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    country = graphene.String()
    sector = graphene.String()
    revenue = graphene.Float()
    ebitda = graphene.Float()
    num_employees = graphene.Int()
    score = graphene.String()
    region = graphene.String()
    description = graphene.String()
    company_type = graphene.String()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
HOST = "https://vpc-datalake-es-dev-pkpwahah2cv4owesrl4g5xzafe.eu-west-2.es.amazonaws.com:443"
INDEX = "company"
client = Elasticsearch(HOST+"/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
embedding_path = "s3://onai-ml-dev-data-eu-west-1/models/peers/embeddings/all_mat.npy"
company_index = "s3://onai-ml-dev-data-eu-west-1/models/peers/embeddings/company_index.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
with smart_open.open(embedding_path, "rb") as f:
    embeddings = np.load(f).astype(np.float32)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
description = pd.read_csv(company_index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
n,d = embeddings.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
index = faiss.IndexFlatL2(d)
index.add(embeddings)
print(index.ntotal)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7581989

In [9]:
company2id = {}
id2company = {}
for i,desc in description.loc[:, ["one_id"]].iterrows():
    company2id[desc["one_id"]] = i
    id2company[i] = desc["one_id"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
public_labels_ciq = {
    2444485: [9429145, 93196, 9870522, 7914436],
    645782: [380011, 392154, 5523392],
    875173: [237655379, 931146, 418171],
    813607: [100231, 357076],
    380011: [357076, 324490, 93339],
    882889: [127202, 5478606, 1025390, 645782],
    46329052: [189096, 915379, 46895276, 877008, 325290136, 20524024, 271958947, 21852987, 26363560, 110104150],
    106335: [319676, 377732, 61206100],
    877769: [874042, 780678, 953488, 883809, 875295, 874186, 874119],
    34049884: [882155, 30428758, 315394, 23037669, 27561],
    254287477: [883300, 30614595],
    5600108: [285880, 5433540, 878697, 35650, 688262, 226852452],
    876031: [410182, 874470, 874191, 879732, 5395336],
    883752: [880697, 65340486, 26320074, 883327, 1034090, 257501324],
    5920885: [1494039, 268074105, 34534627, 20385800, 23000545, 124640],
    628413: [272054403, 91192, 309779, 140283, 138644],
    364040: [381388, 184945, 874170, 42751952, 874183, 314896, 5126590, 841504],
    257501324: [35000, 47320264, 253748612, 85076655, 32053, 12144785, 8186273, 9934160, 557267859],
    695204: [35303, 274561, 683719, 370857, 561001, 874022, 387473, 394038, 8274485, 12188205],
    30614595: [883300, 254287477, 9956099, 380011, 27868703, 2386697, 126857],
    28224119: [26824144, 35023689, 386639, 393661],
    32449506: [875260, 27169270, 5629762, 26014489, 286119],
    233324810: [874864, 159230, 27860587, 35806, 876981],
    879554: [5487000, 236715563, 412090459, 875192, 278679, 180871, 22516334, 30274893, 5478907],
    5580060: [118474533, 1779941, 265154, 10405454],
    23335317: [7885406, 277444, 278933, 8983678, 874143, 409119],
    381865672: [874842, 410366, 873649, 275789, 882473],
    937352: [876758, 879422, 128861678, 6461781, 1859063],
    874119: [881803, 875849, 231533, 877769, 780678, 953488, 875295, 874042, 775001, 874186],
    680934: [135398, 882299, 668578, 4481676, 32012],
    2248076: [141249, 4975204, 98876, 21828553],
    3606442: [882547, 4509042, 20703565, 7435035, 94799, 288033, 359868],
    877235: [295170, 175265, 874520, 410366, 873649, 874977, 167945, 8090046],
    84148802: [275789, 30339992, 5533238, 5718736],
    5523392: [645782, 11809880, 1353107, 962864],
    413744: [409932, 875491, 109303666, 91638],
    314896: [330589, 34768, 184945],
    5126590: [874855, 631781, 364040, 831357, 874170],
    377732: [319676, 106772, 106335, 704634, 320105, 874828, 873861, 1519242, 533853947],
    874652: [377732, 319676, 106772, 704634, 312375, 278933, 874828, 4863668]
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def get_oneid(capiq_id):
    query = client.get(index='company', id=capiq_id)
    return query["_source"]["one_id"]

public_labels = defaultdict(list)
for k,v in public_labels_ciq.items():
    one_id = get_oneid(k)
    for el in v:
        public_labels[one_id].append(get_oneid(el))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
def find_closest_revenue(candidate, revenue, ebitda):
    if candidate.revenue == 0 or candidate.ebitda == 0:
        return candidate.score * 100

    if candidate.revenue < revenue:
        score_revenue = 1 - candidate.revenue/revenue
    else:
        score_revenue = 1 - revenue/candidate.revenue

    if candidate.ebitda < ebitda:
        score_ebitda = 1 - candidate.ebitda/ebitda
    else:
        score_ebitda = 1 - ebitda/candidate.ebitda

    return (score_ebitda + score_revenue) / 2


def revenue_reranker(results, topn=100):
    query = results[0]
    return sorted(results[1:(topn+1)],
                  key=lambda x: find_closest_revenue(x,
                                                     query.revenue,
                                                     query.ebitda
                                                     )
                  )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def query_peers(query_company, result_size=100, reranker=lambda res: res):
    query_vector = embeddings[np.newaxis, company2id[query_company]]

    desc_info = description.loc[company2id[query_company]]
    industry = desc_info["sic_code"]
    country = desc_info["country"]
    
    D, I = index.search(query_vector, result_size)

    ret = []
    added = set()
    for distance,idx in zip(D[0],I[0]):
        id = id2company[idx]
        
        # TODO: fix this to use the pandas dataframe instead
        
        score = distance
        cname = description.loc[idx, "company_name"]
        sector = description.loc[idx, "simple_industry"]
        country = description.loc[idx, "country"]

        ret.append(
            Company(
                id=id,
                name=cname,
                sector=sector,
                country=country,
                score=score,
            )
        )

    return reranker(ret)

def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == company.id:
            return i+1

    return -1


def add_ranks_in_tops(ranks, counts):
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = query_peers(query_id, 500)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['7CE17D1E6E53355C7638AF270390D655', -1, -1, -1, -1]

['81BC07711FBEDE5C0565CF61A01C4DAF', 159, 263, 62]

['AB395747C47AAD43726277D4C69A906C', -1, -1, -1]

['32066D273DA16D21AC90771EDD4BB299', 32, 16]

['3DB2DAA268B964D4A6198DB35BB16BB1', 29, 4, -1]

['40097BD00D04CBAD4312AE1ABC4523BF', 72, -1, -1, 17]

['AA8F66DF7A6EE74C320992CEBC33EBCC', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]

['C42EA57A8F03FA115C8A4671322F7C66', 69, 133, 2]

['FE276F942C9ED6A07430CFDFA19243E1', 393, 10, 211, 17, 26, 56, -1]

['78C55300E19A08F6A09264753956DA4B', 18, -1, -1, -1, 360]

['450A24AAF9E4E2B3FDB28B662F02448C', 309, -1]

['C2A9E8F89D4AE0C73B4958CF86AE6EBC', -1, -1, -1, -1, 449, 97]

['1F7985A0E3B81000F7FCA99CE537C461', 90, 32, -1, -1, -1]

['F03C722062B5503A999CBBFFD00C85FE', -1, -1, -1, -1, -1, -1]

['81245ED573A317A7A218B5625ECD60C5', -1, -1, -1, 2, 49, -1]

['4DC637DAF653DF8C8B22C57FC19EA816', 170, -1, 105, -1, 86]

['3775C12EDE190C736D3D948792EF576B', 36, -1, 20, -1, 270, 33, -1, 206]

['9B89D88E25FB3A8